In [30]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

CONTINENTS = ['Africa', 'Asia', 'Oceania', 'Europe', 'Northern America', 'South America', 'Antarctic Region']
PRIMARY_FORREST_PRODUCTS = ['Roundwood', 'Sawnwood', 'Wood fuel']
PRIMARY_FORREST_PRODUCTS = ['Roundwood', 'Sawnwood', 'Wood-based panels', 'Paper and paperboard']
LAND_USE_FOREST_ITEMS = ['Forestry']

In [31]:
forest_all = pd.read_csv("/home/martin/Documents/school/EPFL/ws/ada/disappearing_forests/data/faostat/Forestry_E_All_Data_cleared.csv", engine='python')


In [32]:
continent_production = forest_all[forest_all.Element.str.match("Production") & 
                                    forest_all.Area.isin(CONTINENTS) &
                                    forest_all.Item.isin(PRIMARY_FORREST_PRODUCTS)]


In [52]:
continent_production[continent_production.Item.str.match("Paper and paperboard")].Unit.unique()

array(['tonnes'], dtype=object)

In [33]:
land_use_all = pd.read_csv("data/faostat/Inputs_LandUse_E_All_Data_cleared.csv", engine='python')
land_use = land_use_all[land_use_all.Item.isin(LAND_USE_FOREST_ITEMS) &
                        land_use_all.Element.str.match('Area') &
                        land_use_all.Area.isin(CONTINENTS)]

In [34]:
year_min = max(continent_production.Year.min(), land_use.Year.min())
year_max = min(continent_production.Year.max(), land_use.Year.max())


In [35]:
continent_production

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
1591725,5100,Africa,1861,Roundwood,5516,Production,1961,1961,m3,277063385.0,A
1591726,5100,Africa,1861,Roundwood,5516,Production,1962,1962,m3,281772967.0,A
1591727,5100,Africa,1861,Roundwood,5516,Production,1963,1963,m3,292837272.0,A
1591728,5100,Africa,1861,Roundwood,5516,Production,1964,1964,m3,294096171.0,A
1591729,5100,Africa,1861,Roundwood,5516,Production,1965,1965,m3,301286184.0,A
...,...,...,...,...,...,...,...,...,...,...,...
1892090,5500,Oceania,1876,Paper and paperboard,5510,Production,2014,2014,tonnes,3810022.0,A
1892091,5500,Oceania,1876,Paper and paperboard,5510,Production,2015,2015,tonnes,3902065.0,A
1892092,5500,Oceania,1876,Paper and paperboard,5510,Production,2016,2016,tonnes,3903474.0,A
1892093,5500,Oceania,1876,Paper and paperboard,5510,Production,2017,2017,tonnes,3966647.0,A


In [36]:
a = continent_production[continent_production.Year == year].groupby(["Area", "Item"]).Value.sum()
a = a.reset_index(level=['Item'])

In [37]:
a

,Item,Value
Area,,
Africa,Paper and paperboard,3.646565e+06
Africa,Roundwood,7.606948e+08
Africa,Sawnwood,9.993484e+06
Africa,Wood-based panels,2.653423e+06
Asia,Paper and paperboard,1.933477e+08
Asia,Roundwood,1.134658e+09
Asia,Sawnwood,1.282555e+08
Asia,Wood-based panels,2.498945e+08
Europe,Paper and paperboard,1.042319e+08


In [59]:
fig = go.Figure()

fig = make_subplots(
    rows=1, 
    cols=2,
    subplot_titles=("Production (m3)", "Forested Area (1000 ha)"),
    shared_yaxes=True
)

for year in range(year_min, year_max):
    a = continent_production[continent_production.Year == year].groupby(["Area", "Item"]).Value.sum()
    a = a.reset_index()
    a = a[a.Area.str.match("Oceania")].append(a[a.Area.str.match("South America")])\
                                      .append(a[a.Area.str.match("Africa")])\
                                      .append(a[a.Area.str.match("Northern America")])\
                                      .append(a[a.Area.str.match("Europe")])\
                                      .append(a[a.Area.str.match("Asia")])
    fig.add_trace(
        go.Bar(
            y=a[a.Item.str.match("Roundwood")].Area.values,
            x=a[a.Item.str.match("Roundwood")].Value.values,
            name='Roundwood, ' + str(year),
            orientation='h',
            visible=False,
        ),    
        row=1,
        col=1, 
    )
    
    fig.add_trace(
        go.Bar(
            y=a[a.Item.str.match("Wood-based panels")].Area.values,
            x=a[a.Item.str.match("Wood-based panels")].Value.values,
            name='Wood-based panels, ' + str(year),
            orientation='h',
            visible=False,
        ),    
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Bar(
            y=a[a.Item.str.match("Sawnwood")].Area.values,
            x=a[a.Item.str.match("Sawnwood")].Value.values,
            name='Sawnwood, ' + str(year),
            orientation='h',
            visible=False,
        ),    
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Bar(
            y=a[a.Item.str.match('Paper and paperboard')].Area.values,
            x=a[a.Item.str.match('Paper and paperboard')].Value.values * 1.2,
            name='Paper and paperboard, ' + str(year),
            orientation='h',
            visible=False,
        ),    
        row=1,
        col=1,
    )

    b = land_use[land_use.Year == year].groupby(["Area", "Item"]).Value.sum()
    b = b.reset_index()
    b = b[b.Area.str.match("Oceania")].append(b[b.Area.str.match("South America")])\
                                      .append(b[b.Area.str.match("Africa")])\
                                      .append(b[b.Area.str.match("Northern America")])\
                                      .append(b[b.Area.str.match("Europe")])\
                                      .append(b[b.Area.str.match("Asia")])
    fig.add_trace(
        go.Bar(
            y=b.Area.values,
            x=b.Value.values,
            name='Forestry, ' + str(year),
            orientation='h',
            visible=False,
        ),
        row=1,
        col=2,
    )
 

In [60]:

fig.data[0].visible = True
fig.data[1].visible = True
fig.data[2].visible = True
fig.data[3].visible = True
fig.data[4].visible = True

steps = []
for i in range(int(len(fig.data)/5)):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
        label='{}'.format(i + year_min),
    )
    step["args"][1][i*5] = True  # Toggle i'th trace to "visible"
    step["args"][1][i*5+1] = True  # Toggle i'th trace to "visible"
    step["args"][1][i*5+2] = True  # Toggle i'th trace to "visible"
    step["args"][1][i*5+3] = True  # Toggle i'th trace to "visible"
    step["args"][1][i*5+4] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=1991,
    currentvalue={"prefix": "Year: "},
    pad={"t": 0},
    steps=steps
)]
fig.update_layout(
    sliders=sliders,
    barmode='stack',
    autosize=False,
    width=1000,
    height=600,
)
fig.update_xaxes(showgrid=True, gridcolor="LightGray")

fig.show()

In [40]:
forest_all[forest_all.Element.str.match("Production") & forest_all.Area.isin(CONTINENTS)].Item.unique()

array(['Roundwood', 'Roundwood:coniferous (production)',
       'Roundwood:non-coniferous (production)', 'Wood fuel',
       'Wood fuel:coniferous', 'Wood fuel:non-coniferous',
       'Industrial roundwood', 'Industrial roundwood:coniferous',
       'Industrial roundwood:non-coniferous', 'Sawlogs and veneer logs',
       'Sawlogs and veneer logs:coniferous',
       'Sawlogs and veneer logs:non-coniferous',
       'Pulpwood:round and split:all species (production)',
       'Pulpwood:round and split:coniferous (production)',
       'Pulpwood:round and split:non-coniferous (production)',
       'Pulpwood and particles (1961-1997)',
       'Pulpwood and particles:coniferous (production:1961-1997)',
       'Pulpwood and particles:non-coniferous (production:1961-1997)',
       'Other industrial roundwood',
       'Other industrial roundwood:coniferous (production)',
       'Other industrial roundwood:non-coniferous (production)',
       'Wood charcoal', 'Wood chips:particles and residues',
 